In [1]:
import nibabel as nib
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from skimage.transform import rotate
from skimage.util import montage
import os
from sklearn.model_selection import train_test_split
import keras
import cv2
import tensorflow as tf
import random
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import numpy as np
from keras.callbacks import CSVLogger
import keras.metrics as K
import tarfile
import pandas as pd


In [2]:
path_to_tar_file = "/kaggle/input/brats-2021-task1/BraTS2021_Training_Data.tar"
output_directory = "brats21-dataset-training-validation"

# Open the tar file in read mode
with tarfile.open(path_to_tar_file, "r") as tar_ref:
    # Extract all contents to the specified directory
    tar_ref.extractall(output_directory)

print(f"Extraction completed. Files are saved to {output_directory}")

Extraction completed. Files are saved to brats21-dataset-training-validation


In [3]:
VOLUME_START_AT = 60 
VOLUME_SLICES = 75 

In [4]:

data_path = "/kaggle/working/brats21-dataset-training-validation"
samples = [f for f in os.listdir(data_path) if not f.startswith('.')]
print("Number of samples:", len(samples))

Number of samples: 1251


In [5]:
# Split the dataset into train and validation sets
samples_train, samples_val = train_test_split(samples, test_size=0.2, random_state=42)

# Split the train set into the real train set and in a test set 
samples_train, samples_test = train_test_split(samples_train, test_size=0.15, random_state=42)

# Print data distribution (Train: 68%, Test: 12%, Val: 20%)
print(f"Train length: {len(samples_train)}")
print(f"Validation length: {len(samples_val)}")
print(f"Test length: {len(samples_test)}")

Train length: 850
Validation length: 251
Test length: 150


In [6]:


IMG_SIZE = 128

from sklearn.utils.class_weight import compute_class_weight

class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, list_IDs, dim=(IMG_SIZE, IMG_SIZE), batch_size=1, n_channels=4, shuffle=True):
        self.dim = dim
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.list_IDs))
        self.on_epoch_end()
        self.class_frequencies = np.zeros(4)  # For 4 classes

    def __len__(self):
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        batch_ids = [self.list_IDs[k] for k in indexes]
        X, y = self.__data_generation(batch_ids)
        return X, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, batch_ids):
        """
        Generates data for a batch of images and corresponding segmentation masks
        :param batch_ids: List of IDs for the current batch
        :return: Input data (X) and one-hot encoded segmentation masks (y)
        """
        # Initialize arrays for the inputs and labels
        X = np.zeros((self.batch_size * VOLUME_SLICES, *self.dim, self.n_channels))  # Updated for 4 channels
        y = np.zeros((self.batch_size * VOLUME_SLICES, *self.dim, 4))  # One-hot encoded masks
    
        # Loop over the batch IDs
        for c, sample_id in enumerate(batch_ids):
            sample_path = os.path.join(data_path, sample_id, sample_id)
            t1ce_path = sample_path + '_t1ce.nii.gz'
            flair_path = sample_path + '_flair.nii.gz'
            seg_path = sample_path + '_seg.nii.gz'
            t1_path = sample_path + '_t1.nii.gz'  # Corrected file extension
            t2_path = sample_path + '_t2.nii.gz'  # Corrected file extension
    
            # Load the images using nibabel
            try:
                t1ce = nib.load(t1ce_path).get_fdata()
                flair = nib.load(flair_path).get_fdata()
                seg = nib.load(seg_path).get_fdata()
                t1 = nib.load(t1_path).get_fdata()
                t2 = nib.load(t2_path).get_fdata()
            except FileNotFoundError:
                print(f"Warning: Files for {sample_id} are missing or cannot be loaded.")
                continue
    
            # Process each slice in the volume (from VOLUME_START_AT to VOLUME_START_AT + VOLUME_SLICES)
            for j in range(VOLUME_SLICES):
                # Resize the input images to the desired dimension
                X[j + VOLUME_SLICES * c, :, :, 0] = cv2.resize(flair[:, :, j + VOLUME_START_AT], self.dim)
                X[j + VOLUME_SLICES * c, :, :, 1] = cv2.resize(t1ce[:, :, j + VOLUME_START_AT], self.dim)
                X[j + VOLUME_SLICES * c, :, :, 2] = cv2.resize(t1[:, :, j + VOLUME_START_AT], self.dim)
                X[j + VOLUME_SLICES * c, :, :, 3] = cv2.resize(t2[:, :, j + VOLUME_START_AT], self.dim)
                
                # Resize the segmentation mask
                seg_slice = seg[:, :, j + VOLUME_START_AT]
    
                # Ensure that the segmentation mask values are within the expected range
                seg_slice = np.clip(seg_slice, 0, 3)  # Clip values to range [0, 3]
    
                # Update the class frequencies
                unique_classes = np.unique(seg_slice)
                for uc in unique_classes:
                    if int(uc) < 4:  # Ensure class is within the expected range
                        self.class_frequencies[int(uc)] += np.sum(seg_slice == uc)
    
                # One-hot encode the segmentation mask
                y[j + VOLUME_SLICES * c] = self.__one_hot_encoding(seg_slice)
    
        # Normalize the input images (if necessary)
        X = X / np.max(X) if np.max(X) > 0 else X
        return X, y

    def __one_hot_encoding(self, mask):
        mask_resized = cv2.resize(mask, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_NEAREST)
        one_hot_mask = np.zeros((IMG_SIZE, IMG_SIZE, 4))
        for i in range(4):
            one_hot_mask[:, :, i] = np.where(mask_resized == i, 1, 0)
        return one_hot_mask

    def repeat(self):
        while True:
            for index in range(len(self)):
                yield self.__getitem__(index)


training_generator = DataGenerator(samples_train)
valid_generator = DataGenerator(samples_val)
test_generator = DataGenerator(samples_test)



In [7]:
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, UpSampling2D, Dropout, concatenate, Input
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
import tensorflow as tf

def build_unet(inputs, ker_init, dropout):
    # Encoder
    conv1 = Conv2D(32, 3, activation='relu', padding='same', kernel_initializer=ker_init,
                   kernel_regularizer=regularizers.l2(0.0001))(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(32, 3, activation='relu', padding='same', kernel_initializer=ker_init,
                   kernel_regularizer=regularizers.l2(0.0001))(conv1)
    conv1 = BatchNormalization()(conv1)

    pool = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer=ker_init,
                  kernel_regularizer=regularizers.l2(0.0001))(pool)
    conv = BatchNormalization()(conv)
    conv = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer=ker_init,
                  kernel_regularizer=regularizers.l2(0.0001))(conv)
    conv = BatchNormalization()(conv)

    pool1 = MaxPooling2D(pool_size=(2, 2))(conv)
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer=ker_init,
                   kernel_regularizer=regularizers.l2(0.0001))(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer=ker_init,
                   kernel_regularizer=regularizers.l2(0.0001))(conv2)
    conv2 = BatchNormalization()(conv2)

    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer=ker_init,
                   kernel_regularizer=regularizers.l2(0.0001))(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer=ker_init,
                   kernel_regularizer=regularizers.l2(0.0001))(conv3)
    conv3 = BatchNormalization()(conv3)

    pool4 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv5 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer=ker_init,
                   kernel_regularizer=regularizers.l2(0.0001))(pool4)
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer=ker_init,
                   kernel_regularizer=regularizers.l2(0.0001))(conv5)
    conv5 = BatchNormalization()(conv5)
    drop5 = Dropout(dropout)(conv5)

    # Decoder
    up7 = Conv2D(256, 2, activation='relu', padding='same', kernel_initializer=ker_init,
                 kernel_regularizer=regularizers.l2(0.0001))(UpSampling2D(size=(2, 2))(drop5))
    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer=ker_init,
                   kernel_regularizer=regularizers.l2(0.0001))(merge7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer=ker_init,
                   kernel_regularizer=regularizers.l2(0.0001))(conv7)
    conv7 = BatchNormalization()(conv7)

    up8 = Conv2D(128, 2, activation='relu', padding='same', kernel_initializer=ker_init,
                 kernel_regularizer=regularizers.l2(0.0001))(UpSampling2D(size=(2, 2))(conv7))
    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer=ker_init,
                   kernel_regularizer=regularizers.l2(0.0001))(merge8)
    conv8 = BatchNormalization()(conv8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer=ker_init,
                   kernel_regularizer=regularizers.l2(0.0001))(conv8)
    conv8 = BatchNormalization()(conv8)

    up9 = Conv2D(64, 2, activation='relu', padding='same', kernel_initializer=ker_init,
                kernel_regularizer=regularizers.l2(0.0001))(UpSampling2D(size=(2, 2))(conv8))
    merge9 = concatenate([conv, up9], axis=3)
    conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer=ker_init,
                   kernel_regularizer=regularizers.l2(0.0001))(merge9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer=ker_init,
                   kernel_regularizer=regularizers.l2(0.0001))(conv9)
    conv9 = BatchNormalization()(conv9)

    up = Conv2D(32, 2, activation='relu', padding='same', kernel_initializer=ker_init,
                kernel_regularizer=regularizers.l2(0.0001))(UpSampling2D(size=(2, 2))(conv9))
    merge = concatenate([conv1, up], axis=3)
    conv = Conv2D(32, 3, activation='relu', padding='same', kernel_initializer=ker_init,
                  kernel_regularizer=regularizers.l2(0.0001))(merge)
    conv = BatchNormalization()(conv)
    conv = Conv2D(32, 3, activation='relu', padding='same', kernel_initializer=ker_init,
                  kernel_regularizer=regularizers.l2(0.0001))(conv)
    conv = BatchNormalization()(conv)

    # Output Layer
    conv10 = Conv2D(4, 1, activation='softmax')(conv)

    return Model(inputs=inputs, outputs=conv10)

In [8]:
import tensorflow as tf
import tensorflow.keras.backend as K

def dice_coef(y_true, y_pred, smooth=1):
    """
    Dice coefficient for measuring the similarity between predicted and true segmentation masks.
    :param y_true: Ground truth segmentation mask
    :param y_pred: Predicted segmentation mask
    :param smooth: Smoothing factor to avoid division by zero
    :return: Dice coefficient
      """
    dice_coefficient = 0.0

    for i in range(4):  # Fixed to 4 classes
        y_true_f = tf.cast(tf.reshape(y_true[..., i], [-1]), tf.float32)
        y_pred_f = tf.cast(tf.reshape(y_pred[..., i], [-1]), tf.float32)

        # Compute intersection and union
        intersection = K.sum(y_true_f * y_pred_f)
        dice_coefficient += (2.0 * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

    # Average Dice coefficient across 4 classes
    dice_coefficient /= 4.0
    return dice_coefficient 

def precision(y_true, y_pred):
    
    y_true = K.cast(y_true, dtype='float32')
    y_pred = K.cast(y_pred, dtype='float32')

    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    
    return precision

def sensitivity(y_true, y_pred):
    
    y_true = K.cast(y_true, dtype='float32')
    y_pred = K.cast(y_pred, dtype='float32')

    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

def specificity(y_true, y_pred):
    
    y_true = K.cast(y_true, dtype='float32')
    y_pred = K.cast(y_pred, dtype='float32')

    true_negatives = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1 - y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())


In [9]:

counts = np.array([8868079, 11639, 27873, 20409])


class_weights = 1. / counts
class_weights /= np.sum(class_weights)  

print("Class weights:", class_weights)

Class weights: [0.0006598  0.50272157 0.20992274 0.28669589]


In [10]:
def weighted_categorical_crossentropy(class_weights):
    class_weights = tf.convert_to_tensor(class_weights, dtype=tf.float32)
    def loss(y_true, y_pred):

        loss = tf.reduce_sum(-y_true * tf.math.log(y_pred + 1e-7), axis=-1)      
        
        weighted_loss = loss * tf.gather(class_weights, tf.argmax(y_true, axis=-1))

        return tf.reduce_mean(weighted_loss)

    return loss

In [11]:
from sklearn.utils.class_weight import compute_class_weight
# Define input data shape
input_layer = Input((IMG_SIZE, IMG_SIZE, 4))

# Build and compile the model
model = build_unet(input_layer, 'he_normal', 0.3)

model.compile(
    loss=weighted_categorical_crossentropy(class_weights),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001, clipvalue=1.0),  # Gradient clipping
    
    metrics=[
        'accuracy',
        tf.keras.metrics.MeanIoU(num_classes=4),
        dice_coef,
        precision,
        sensitivity,
        specificity,
    ],
)

In [12]:
callbacks = [
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001, verbose=1
    ),
    
    keras.callbacks.ModelCheckpoint(
        filepath='model_.{epoch:02d}-{val_loss:.6f}.weights.h5',
        verbose=1,
        save_best_only=True,
        save_weights_only=True
    ),
    
    CSVLogger('training.log', separator=',', append=False)
]


In [13]:

steps_per_epoch = len(training_generator)
validation_steps = len(valid_generator)

# Ensure generators are infinite
training_generator = training_generator.repeat()
valid_generator = valid_generator.repeat()
# Train the model
model.fit(
    training_generator,
    validation_data=valid_generator,
    epochs=10,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=callbacks,
    verbose=1  
)


Epoch 1/10


I0000 00:00:1734175857.469944      94 service.cc:145] XLA service 0x785ff8001580 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1734175857.470702      94 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1734175857.470712      94 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1734175905.802981      94 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


850/850 ━━━━━━━━━━━━━━━━━━━━ 0s 854ms/step - accuracy: 0.5168 - dice_coef: 0.1349 - loss: 0.4086 - mean_io_u: 0.2512 - precision: 0.3261 - sensitivity: 0.1306 - specificity: 0.9123
Epoch 1: val_loss improved from inf to 0.06928, saving model to model_.01-0.069279.weights.h5
850/850 ━━━━━━━━━━━━━━━━━━━━ 1005s 1s/step - accuracy: 0.5169 - dice_coef: 0.1349 - loss: 0.4084 - mean_io_u: 0.2512 - precision: 0.3260 - sensitivity: 0.1306 - specificity: 0.9123 - val_accuracy: 0.7626 - val_dice_coef: 0.1521 - val_loss: 0.0693 - val_mean_io_u: 0.2500 - val_precision: 0.2641 - val_sensitivity: 0.0610 - val_specificity: 0.9396 - learning_rate: 1.0000e-04
Epoch 2/10
850/850 ━━━━━━━━━━━━━━━━━━━━ 0s 850ms/step - accuracy: 0.7290 - dice_coef: 0.1560 - loss: 0.0515 - mean_io_u: 0.2512 - precision: 0.3202 - sensitivity: 0.1032 - specificity: 0.9300
Epoch 2: val_loss improved from 0.06928 to 0.03171, saving model to model_.02-0.031711.weights.h5
850/850 ━━━━━━━━━━━━━━━━━━━━ 937s 1s/step - accuracy: 0.7291

In [14]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 4)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 128, 128,  │      1,184 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 128, 128,  │        128 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 128, 128,  │      9,248 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        128 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 64, 64,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 64, 64,    │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │        256 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 64, 64,    │     36,928 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │        256 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 32, 32,    │     73,856 │ max_pooling2d_1[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        512 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 32, 32,    │    147,584 │ batch_normalizat… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        512 │ conv2d_5[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 16, 16,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 16, 16,    │    295,168 │ max_pooling2d_2[

 Total params: 23,305,006 (88.90 MB)

 Trainable params: 7,766,372 (29.63 MB)

 Non-trainable params: 5,888 (23.00 KB)

 Optimizer params: 15,532,746 (59.25 MB)